In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


In [2]:
from google import genai
from google.genai import types


In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [4]:
from google import generativeai as genai
from google.generativeai import types
from kaggle_secrets import UserSecretsClient
import json
import numpy as np
from sentence_transformers import SentenceTransformer, util


2025-04-21 04:58:58.594687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745211538.827845      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745211538.895556      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
few_shot_prompt = """
User: I want to book a flight from New York to Paris, leaving June 1 and returning June 10, economy class.
AI:
{
  "origin": "New York",
  "destination": "Paris",
  "departure_date": "2025-06-01",
  "return_date": "2025-06-10",
  "class": "Economy",
  "price_range": "$500-$700"
}

User: Book a ticket from Mumbai to London for August 15, business class.
AI:
{
  "origin": "Mumbai",
  "destination": "London",
  "departure_date": "2025-08-15",
  "return_date": null,
  "class": "Business",
  "price_range": "$1200-$1500"
}
"""

# =============================
# 🧠 Travel FAQ Knowledge Base (for RAG)
# =============================
travel_docs = [
    {"question": "What is the baggage policy?", "answer": "Economy class allows 1 checked bag (23kg)."},
    {"question": "What is the refund policy?", "answer": "Tickets are refundable only if insurance is purchased."},
    {"question": "What are the flight change fees?", "answer": "Changes cost $150 plus fare difference."}
]

In [6]:

# Encode FAQ questions
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
faq_embeddings = embed_model.encode([doc['question'] for doc in travel_docs], convert_to_tensor=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def book_ticket(details):
    return f"\n🎟️ Ticket booked! {details['origin']} ➡️ {details['destination']} on {details['departure_date']} ({details['class']} Class)"

# =============================
# ✉️ Function: FAQ Retrieval
# =============================
def retrieve_faq_answer(query):
    query_embedding = embed_model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, faq_embeddings)
    best_idx = int(np.argmax(scores))
    return travel_docs[best_idx]['answer']


In [8]:
user_input = "Book me a flight from Boston to Tokyo on September 10, returning September 20, economy class."

# Create prompt
prompt = f"""
{few_shot_prompt}

User: {user_input}
AI:
"""

In [9]:

response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=[prompt]
)


In [10]:


# =============================
# 📦 Parse Gemini Output
# =============================
try:
    flight_details = json.loads(response.candidates[0].content.parts[0].text)
except json.JSONDecodeError:
    print("⚠️ Model output not JSON parsable. Raw output:")
    print(response.candidates[0].content.parts[0].text)
    flight_details = {}

# =============================
# ✅ Book Ticket if JSON is Valid
# =============================
if flight_details:
    booking_confirmation = book_ticket(flight_details)
    print(booking_confirmation)

# =========

⚠️ Model output not JSON parsable. Raw output:
```json
{
  "origin": "Boston",
  "destination": "Tokyo",
  "departure_date": "2025-09-10",
  "return_date": "2025-09-20",
  "class": "Economy",
  "price_range": "$800-$1000"
}
``` 



In [11]:
user_input = "Book me a flight from Vizag to Banglore on Jan 24, returning September 20, business class."

# Create prompt
prompt = f"""
{few_shot_prompt}

User: {user_input}
AI:
"""

response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=[prompt]
)


In [12]:
import re

raw_text = response.candidates[0].content.parts[0].text

# Strip markdown ```json ... ``` if present
cleaned_text = re.sub(r"```json|```", "", raw_text).strip()

try:
    flight_details = json.loads(cleaned_text)
except json.JSONDecodeError:
    print("⚠️ Model output not JSON parsable. Cleaned raw output:")
    print(cleaned_text)
    flight_details = {}

In [13]:
if flight_details:
    booking_confirmation = book_ticket(flight_details)
    print(booking_confirmation)


🎟️ Ticket booked! Vizag ➡️ Bangalore on 2024-01-24 (Business Class)
